# 32. 管理属性（Managing Attributes）

管理属性让对象“字段访问”更安全：property 用于校验/派生属性；__getattr__/__getattribute__ 用于拦截访问；__slots__ 可限制属性并节省内存。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 22 节：类基础


## 知识点地图

- 1. property：方法变属性（带校验）
- 2. __getattr__：属性不存在时的兜底
- 3. __getattribute__（了解）：拦截所有属性访问
- 4. __slots__：限制属性并节省内存（了解）
- 5. 描述符（了解）：property 本质上是描述符


## 自检清单（学完打勾）

- [ ] 会用 @property 把方法包装成属性访问
- [ ] 会写 setter 做输入校验
- [ ] 理解 __getattr__：找不到属性时兜底
- [ ] 了解 __getattribute__：所有属性访问都会走（谨慎）
- [ ] 了解 __slots__ 的用途与限制


## 知识点 1：property：方法变属性（带校验）

property 常用于：校验输入、计算派生属性、保持对外接口稳定。


In [ ]:
class Celsius:
    def __init__(self, value):
        self._value = float(value)

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, v):
        v = float(v)
        if v < -273.15:
            raise ValueError('below absolute zero')
        self._value = v

c = Celsius(0)
print(c.value)
c.value = 100
print(c.value)


## 知识点 2：__getattr__：属性不存在时的兜底

__getattr__ 仅在正常查找失败时调用；用于代理/动态属性。必须抛 AttributeError。


In [ ]:
class DictObj:
    def __init__(self, d):
        self._d = dict(d)

    def __getattr__(self, name):
        try:
            return self._d[name]
        except KeyError as e:
            raise AttributeError(name) from e

obj = DictObj({'a': 1})
print(obj.a)


## 知识点 3：__getattribute__（了解）：拦截所有属性访问

__getattribute__ 每次访问属性都会触发，容易写出递归错误；只有必要时才用。


In [ ]:
class Demo:
    def __init__(self):
        self.x = 1
    def __getattribute__(self, name):
        # 注意：必须用 super().__getattribute__ 取值
        print('get', name)
        return super().__getattribute__(name)

d = Demo()
print(d.x)


## 知识点 4：__slots__：限制属性并节省内存（了解）

__slots__ 让实例不能随意加新属性，也可能减少内存占用。缺点是灵活性降低。


In [ ]:
class Slim:
    __slots__ = ('x', 'y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

s = Slim(1, 2)
print(s.x, s.y)
try:
    s.z = 3
except AttributeError as e:
    print('no z:', e)


## 知识点 5：描述符（了解）：property 本质上是描述符

理解到这里即可：descriptor 是更通用的“托管属性”机制（进阶主题）。


## 常见坑

- __getattribute__ 容易写出无限递归（一定用 super().__getattribute__）
- __getattr__ 必须抛 AttributeError（否则很多内置逻辑会异常）


## 综合小案例：实现 User.age：用 property 校验非负整数

写 User(age)：age 只能是非负 int；否则抛 ValueError。


In [ ]:
class User:
    def __init__(self, age):
        self.age = age

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, v):
        if not isinstance(v, int) or isinstance(v, bool):
            raise ValueError('age must be int')
        if v < 0:
            raise ValueError('age must be >= 0')
        self._age = v

u = User(18)
print(u.age)
try:
    u.age = -1
except Exception as e:
    print(type(e).__name__, e)


## 自测题（不写代码也能回答）

- property 适合解决哪些问题？
- __getattr__ 与 __getattribute__ 的调用时机有何不同？
- __slots__ 带来哪些优缺点？


## 练习题（建议写代码）

- 实现一个只读配置对象：对已有字段只读（setter 抛异常），不存在字段抛 AttributeError。
- 实现一个派生属性 full_name：由 first_name/last_name 计算得到。
